In [1]:
library(data.table)
library(repr)
library(lubridate)
library(rpart)
library(partykit)
library(ggplot2)
library(Metrics)
library(TSdist)
library(dtw)
library(TSrepr)
library(TunePareto)
library(caret)
options(repr.plot.width=10, repr.plot.height=4)

Warning message:
"package 'data.table' was built under R version 3.6.3"
Attaching package: 'lubridate'

The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following object is masked from 'package:base':

    date

Warning message:
"package 'partykit' was built under R version 3.6.3"Loading required package: grid
Loading required package: libcoin
Warning message:
"package 'libcoin' was built under R version 3.6.3"Loading required package: mvtnorm
Warning message:
"package 'mvtnorm' was built under R version 3.6.3"Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Warning message:
"package 'Metrics' was built under R version 3.6.3"Warning message:
"package 'TSdist' was built under R version 3.6.3"Loading required package: proxy
Warning message:
"package 'proxy' was built under R version 3.6.3"
Attaching

In [2]:
current_folder = getwd()
dat = fread('YAT-01012019-10122021.csv')
setnames(dat,names(dat),c('date','hour','yat_one','yat_two','yat_three'))
dat[,datex:=strptime(date,'%d/%m/%Y')]
dat[,tst:=ymd_hm(paste(datex,hour))]
dat[,date:=date(tst)]
dat[,hour:=hour(tst)]
dat[,yat_one_t:=gsub('\\.','',yat_one)]
dat[,yat_two_t:=gsub('\\.','',yat_two)]
dat[,yat_three_t:=gsub('\\.','',yat_three)]
dat[,yat_one:=as.numeric(gsub(',','.',yat_one_t))]
dat[,yat_two:=as.numeric(gsub(',','.',yat_two_t))]
dat[,yat_three:=as.numeric(gsub(',','.',yat_three_t))]
yat_dat = dat[,list(date,hour,yat_one,yat_two,yat_three)]
dat=fread('YAL-01012019-10122021.csv')
setnames(dat,names(dat),c('date','hour','yat_one','yat_two','yat_three'))
dat[,datex:=strptime(date,'%d/%m/%Y')]
dat[,tst:=ymd_hm(paste(datex,hour))]
dat[,date:=date(tst)]
dat[,hour:=hour(tst)]
dat[,yat_one_t:=gsub('\\.','',yat_one)]
dat[,yat_two_t:=gsub('\\.','',yat_two)]
dat[,yat_three_t:=gsub('\\.','',yat_three)]
dat[,yal_one:=as.numeric(gsub(',','.',yat_one_t))]
dat[,yal_two:=as.numeric(gsub(',','.',yat_two_t))]
dat[,yal_three:=as.numeric(gsub(',','.',yat_three_t))]
yal_dat = dat[,list(date,hour,yal_one,yal_two,yal_three)]
final_dat = merge(yat_dat,
yal_dat,
by=c("date","hour"),
all=FALSE)
total_vol = final_dat[, list(yat_vol = yat_one + yat_two + yat_three,
yal_vol = yal_one + yal_two + yal_three,
net_imb = (yat_one + yat_two + yat_three) - (yal_one
+ yal_two + yal_three)), by=c("date", "hour")]
total_vol[, direction:=ifelse(net_imb>50, 1 , ifelse(net_imb<(-50),-1,0))]
total_vol[, net_imb:=ifelse(net_imb<(-5000), (-5000), ifelse(net_imb>5000, 5000, net_imb))]

Warning message in strptime(date, "%d/%m/%Y"):
"strptime() usage detected and wrapped with as.POSIXct(). This is to minimize the chance of assigning POSIXlt columns, which use 40+ bytes to store one date (versus 8 for POSIXct). Use as.POSIXct() (which will call strptime() as needed internally) to avoid this warning."Warning message in strptime(date, "%d/%m/%Y"):
"strptime() usage detected and wrapped with as.POSIXct(). This is to minimize the chance of assigning POSIXlt columns, which use 40+ bytes to store one date (versus 8 for POSIXct). Use as.POSIXct() (which will call strptime() as needed internally) to avoid this warning."

In [4]:
diff <- total_vol$net_imb - shift(total_vol$net_imb,1)
diff[1] <- mean(diff[2:length(diff)])
total_vol[,diff:=diff]

In [ ]:
total_vol[,t:=1:.N]
lm_tree <- lmtree(net_imb~t | t,total_vol)
pred_lmtree <- predict(lm_tree,total_vol)
total_vol$lmtree <- pred_lmtree
save(total_vol,file="C:/Users/kaan9/OneDrive/Masaüstü/total_vol.RData")

In [7]:
load("C:/Users/kaan9/OneDrive/Masaüstü/total_vol.RData")

In [6]:
series <- list()
start_hours_before <- 6
window_sizes <- c(12,24,36)
total_vol$raw <- total_vol$net_imb
cols <- c("raw","diff","lmtree")
raw_names <- character(0)
hour_window <- character(0)
for(c in cols){
    for(w in window_sizes){
        tmp <- data.table(total_vol)
        tmp[, paste0((start_hours_before):(w+start_hours_before-1), "_hours_before") := shift(tmp[[c]], (start_hours_before):(w+start_hours_before-1))]
        tmp <- tmp[complete.cases(tmp),]
        for(h in 12:23){
           hour <- paste("hour",as.character(h),sep="")
           window <- paste("window",as.character(w),sep="")
           st <- which(colnames(tmp)==paste(start_hours_before,"hours_before",sep="_"))
           e <- length(colnames(tmp))
           dir <- which(colnames(tmp)=="direction") 
           indices <- c(1,dir,st:e)
           series[[paste(hour,window,c,sep="_")]]  <- tmp[hour==h,..indices]
           if(c=="raw"){
               raw_names <- c(raw_names,paste(hour,window,c,sep="_"))
               hour_window <- c(hour_window,paste(hour,window,sep="_"))
           } 
        }
    }
}

In [9]:
time_start <- Sys.time()
for(i in 1:length(raw_names)){
  series_long <- melt(series[[raw_names[i]]],id.vars = c("date","direction"))
  long_dt <- data.table()
  for(d in unique(series_long$date)){
    temp_dt <- series_long[date==d,]
    temp_tree <- rpart(value~variable,temp_dt,minbucket=1,minsplit=2)
    temp_pred <- predict(temp_tree)
    temp_dt$tree <- temp_pred
    temp_dt[,t:=1:.N]
    temp_sax <- repr_sax(temp_dt$value, q = 2, a = 4)
    dummy_time=c(1:(length(temp_sax)-1))*2
    dummy_time=c(dummy_time,nrow(temp_dt))  
    dt_sax=data.table(t=dummy_time,sax_rep_char=temp_sax)
    temp_dt <- merge(temp_dt,dt_sax,by="t",all.x=T)
    temp_dt[,sax_rep_char_num:=nafill(as.numeric(as.factor(sax_rep_char)),'nocb')] # from data.table  
    temp_dt[,sax_rep:=mean(value),by=list(sax_rep_char_num)]  
    long_dt <- rbind(long_dt,temp_dt)
  }
  series[[paste(hour_window[i],"tree",sep="_")]] <- dcast(long_dt,date+direction~variable,value.var="tree")
  series[[paste(hour_window[i],"sax",sep="_")]] <- dcast(long_dt,date+direction~variable,value.var="sax_rep")  
}
save(series,file="C:/Users/kaan9/OneDrive/Masaüstü/series.RData")
time_end <- Sys.time()

In [10]:
time_end-time_start

Time difference of 6.013768 mins

In [15]:
load("C:/Users/kaan9/OneDrive/Masaüstü/series.RData")

In [4]:
time_start1 <- Sys.time()
distances <- list()
for(n in names(series)[c(1,13,25)]){
 
    distances[[paste(n,"euc",sep="_")]] <- as.matrix(dist(series[[n]][,3:length(series[[n]])]))
    diag(distances[[paste(n,"euc",sep="_")]]) <- 10000
    
    distances[[paste(n,"dtw",sep="_")]] <- as.matrix(dtwDist(series[[n]][(365*2):nrow(series[[n]]),3:length(series[[n]])],window.type='sakoechiba',window.size=10))
    diag(distances[[paste(n,"dtw",sep="_")]]) <- 10000
    
    distances[[paste(n,"edr",sep="_")]] <- as.matrix(TSDatabaseDistances(series[[n]][(365*2):nrow(series[[n]]),3:length(series[[n]])],distance='erp',g=0.5))
    diag(distances[[paste(n,"edr",sep="_")]]) <- 10000
    
}
save(distances,file="C:/Users/kaan9/OneDrive/Masaüstü/distances.RData")
time_end1 <- Sys.time()

In [11]:
time_start1 <- Sys.time()
distances <- list()
for(n in names(series)){
 
    distances[[paste(n,"euc",sep="_")]] <- as.matrix(dist(series[[n]][,3:length(series[[n]])]))
    diag(distances[[paste(n,"euc",sep="_")]]) <- 10000
    
}
save(distances,file="C:/Users/kaan9/OneDrive/Masaüstü/distances.RData")
time_end1 <- Sys.time()

In [3]:
load("C:/Users/kaan9/OneDrive/Masaüstü/distances.RData")

In [12]:
time_end1 - time_start1

Time difference of 1.950815 mins

In [30]:
set.seed(131)
cv_indices <- list()
trainclasses <- list()
for(n in names(series)){
    
    trainclasses[[n]] <- series[[n]]$direction
    cv_indices[[n]] <- generateCVRuns(trainclasses[[n]], ntimes =5, nfold = 10, 
                                      leaveOneOut = FALSE, stratified = TRUE)
}

In [5]:
nn_classify_cv=function(dist_matrix,train_class,test_indices,k=1){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]
    #print(str(test_distances_to_train))
    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(train_class[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {train_class[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')
    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}

In [ ]:
time_starta <- Sys.time()
k_levels <- c(1,3,5)
results <- data.table(hour=character(0),model=character(0),mean_acc=numeric(0),sd_acc=numeric(0))
for(matr in names(distances)){
    hour <- substr(matr,start = 1,stop=gregexpr(pattern ='_',matr)[[1]][1]-1)
    name <- substr(matr,start = 1,stop=gregexpr(pattern ='_',matr)[[1]][3]-1)
    for(k in k_levels){
        k_p <- paste("k",as.character(k),sep="")
        acc_temp <- numeric(0)
        for(rep in 1:5){
            this_rep <- cv_indices[[name]][[rep]]
            for(fold in 1:10){
                test_indices <- this_rep[[fold]]
                preds <- nn_classify_cv(distances[[matr]],trainclasses[[name]],
                                        test_indices,k)$prediction$predicted
                acc_temp <- c(acc_temp,
                              accuracy(trainclasses[[name]][test_indices],preds))
            }
        }
        results <- rbind(results,list(hour,paste(matr,k_p,sep="_"),mean(acc_temp),sd(acc_temp)))
    }
}
time_enda <- Sys.time()

In [51]:
time_enda-time_starta

Time difference of 3.254543 secs

In [47]:
results

hour,model,mean_acc,sd_acc
hour12,hour12_window12_raw_euc_k1,0.5317290,0.04512681
hour12,hour12_window12_raw_euc_k3,0.5497594,0.04319266
hour12,hour12_window12_raw_euc_k5,0.5808809,0.03630440
